In [ ]:
import os
from PIL import Image
import util.image_import as ii
from glob import glob
import numpy as np
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
import matplotlib.pyplot as plt
from keras.applications import mobilenet
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras import models
from keras import layers
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam, RMSprop, SGD
from util.plots import plot_history
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Subtracting mean and normalizing to save pictures

images_pot        = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/potato", "*.jpg"))
images_cat        = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/catfood", "*.jpg"))
images_tab        = glob(os.path.join("/home/mikkel/Documents/github/BSc-PRO/Classification/res_still/train/table", "*.jpg"))

X_train = ii.images_import(images_pot, images_cat, images_tab)

mean_image, train_pot = ii.make_images_mean(X_train, "res_still", "res_still_mean")


In [2]:
# Importing the images
train_pot = glob(os.path.join("../Classification/res_still/train/potato", "*.jpg"))
train_cat = glob(os.path.join("../Classification/res_still/train/catfood", "*.jpg"))
train_tab = glob(os.path.join("../Classification/res_still/train/table", "*.jpg"))
val_pot   = glob(os.path.join("../Classification/res_still/valid/potato", "*.jpg"))
val_cat   = glob(os.path.join("../Classification/res_still/valid/catfood", "*.jpg"))
val_tab   = glob(os.path.join("../Classification/res_still/valid/table", "*.jpg"))
test_pot  = glob(os.path.join("../Classification/res_still/test/potato", "*.jpg"))
test_cat  = glob(os.path.join("../Classification/res_still/test/catfood", "*.jpg"))
test_tab  = glob(os.path.join("../Classification/res_still/test/table", "*.jpg"))

X_train, y_train = ii.images_to_numpy(train_pot, train_cat, train_tab)
X_val, y_val = ii.images_to_numpy(val_pot, val_cat, val_tab)
X_test, y_test = ii.images_to_numpy(test_pot, test_cat, test_tab)

batch_size = 32
num_classes = 3

# Making datagen for training with normilization, center, and dataugmentation
train_datagen = ImageDataGenerator(
        featurewise_center=True,
        #featurewise_std_normalization=True
        rescale=1./255,
        rotation_range=10,
        vertical_flip=True,
        horizontal_flip=True,
        brightness_range=(0.65, 1.35)
)

# Making datagen for validation and test with normilization
valid_datagen = ImageDataGenerator(
        featurewise_center=True, 
        #featurewise_std_normalization=True
        rescale=1./255
)


X_train = X_train.astype(float)
X_val = X_val.astype(float)
X_test = X_test.astype(float)

y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

# Required for featurewise normalization
train_datagen.fit(X_train/255.)
valid_datagen.fit(X_train/255.)

train_generator = train_datagen.flow(X_train,
                                     y_train,
                                     batch_size=batch_size, shuffle=True)

validation_generator = valid_datagen.flow(X_val,
                                     y_val,
                                     batch_size=batch_size, shuffle=True)

test_generator = valid_datagen.flow(X_test,
                                     y_test,
                                     batch_size=batch_size, shuffle=True)


## Optimized model

In [3]:
model = Sequential()
model.add(Flatten(input_shape=(224,224,3)))
# kernel_regularizer: Regularizer function applied to the kernel weights matrix (see regularizer).
model.add(Dense(128 ,kernel_regularizer=regularizers.l2(0.02)))
#model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(3, kernel_regularizer=regularizers.l2(0.02)))
#model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.00001, decay=0.0), # lr=0.0001
              metrics=['accuracy'])

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               19267712  
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 19,268,099
Trainable params: 19,268,099
Non-trainable params: 0
_________________________________________________________________


In [4]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=384 // batch_size,
            epochs=100,
            validation_data=validation_generator,
            validation_steps=100 // batch_size)
plot_history(history, mean_N=1)

Epoch 1/100
12/12 [==============================] - 4s 312ms/step - loss: 6.2945 - acc: 0.4474 - val_loss: 6.3090 - val_acc: 0.4271
Epoch 2/100
12/12 [==============================] - 3s 270ms/step - loss: 6.3893 - acc: 0.4216 - val_loss: 6.1785 - val_acc: 0.5588
Epoch 3/100
12/12 [==============================] - 3s 282ms/step - loss: 6.2929 - acc: 0.4219 - val_loss: 6.1973 - val_acc: 0.3088
Epoch 4/100
 7/12 [================>.............] - ETA: 1s - loss: 6.2145 - acc: 0.3661

KeyboardInterrupt: 

In [ ]:
imgs, labels = next(test_generator)

#ynew = model.predict(imgs)


model.evaluate(imgs,labels, batch_size=batch_size)

In [ ]:
imgs, labels = next(validation_generator)
model.predict(imgs)
#print(imgs)

## Hyperparameter optimization

In [ ]:
# Hyperparameters
learning_rates = [1e-3,1e-4, 1e-5];
regulization_strengths = [0, 0.2, 0.02];
learning_decay = [0, 0.1, 0.01];
history = {}

for lr in learning_rates:
    for r in regulization_strengths:
        for d in learning_decay:

        # Defining the model:
        model = Sequential()
        model.add(Flatten(input_shape=(224,224,3)))
        # kernel_regularizer: Regularizer function applied to the kernel weights matrix (see regularizer).
        model.add(Dense(128 ,kernel_regularizer=regularizers.l2(r)))
        model.add(Activation('relu'))
        model.add(Dense(3, kernel_regularizer=regularizers.l2(r)))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=lr, decay=d), # lr=0.0001
                      metrics=['accuracy'])
        # Running the model:
        history.append(model.fit_generator(
            train_generator,
            steps_per_epoch=384 // batch_size,
            epochs=100,
            validation_data=validation_generator,
            validation_steps=100 // batch_size))

True
